<a href="https://colab.research.google.com/github/zahra370/Covid19/blob/main/lungsegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pydicom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 7.5 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from pydicom import dcmread
import cv2 as ocv
import pydicom
from sklearn.cluster import KMeans
from skimage import measure, morphology
from PIL import Image ,ImageFilter, ImageEnhance

from ipywidgets import Layout, interact, IntSlider

from skimage.segmentation import clear_border
################################################
from scipy import ndimage as ndi
from skimage import morphology
from skimage.measure import label, regionprops



In [ ]:
class scan:
    def __init__(self, path):
        self.path = path
        self.scan_stack = self.__get_stack__()
    
    def __load_slices__(self):
        slices = [pydicom.read_file(self.path + '/' + s) for s in os.listdir(self.path)]
        slices.sort(key = lambda x: float(x.ImagePositionPatient[2]),reverse = True)
        try:
          slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
        except:
          slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
        for s in slices:
          s.SliceThickness = slice_thickness
        
        return slices
    
    def __get_stack__(self):
        slices = self.__load_slices__()
        img = np.stack([s.pixel_array for s in slices])
        img = img.astype(np.int16)
        img[img == -2000] = 0
        intercept = slices[0].RescaleIntercept
        slope = slices[0].RescaleSlope
        if slope != 1:
            img = slope * img.astype(np.float64)
            img = img.astype(np.int16)
        img += np.int16(intercept)
        return np.array(img, dtype=np.int16)
    
    def get_mask(self, idx):
        img = self.scan_stack[idx]
        binary_image = img <= -320 
        labels = measure.label(binary_image)
        background_label = labels[0,0]
        
        clear_bw = clear_border(binary_image)
        eroded = morphology.erosion(clear_bw,np.ones([3,3]))
        dilation = morphology.dilation(clear_bw,np.ones([11,11]))
        cleaned = morphology.remove_small_objects(dilation, 200)
        # erosion
 
        mask = eroded * img
        
        return img, binary_image, clear_bw, dilation, cleaned, mask

In [ ]:
sample = scan('/content/drive/MyDrive/DICOM')

In [ ]:
def printer(x):
    img, binary_image, clear_bw, dilation, cleaned, mask = sample.get_mask(x)
    # image = applied_mask.astype("uint8")



    ########################################
    #  watershed

    
    from skimage import filters, feature, morphology , segmentation, color
    from skimage.color import label2rgb 
    from scipy.ndimage import distance_transform_edt

    simg = np.array(mask, dtype = float) 
    simg = (simg - simg.min()) / (simg.max() - simg.min()) * 255.0  
    simg = simg.astype(np.uint8)



    ########################################
    # mask = np.array(raw_mask, dtype = float) 
    # mask= (mask - mask.min()) / (mask.max() - mask.min()) * 255.0  
    # mask = mask.astype(np.uint8)
    from skimage.segmentation import clear_border
    # clahe = ocv.createCLAHE(clipLimit =3.0, tileGridSize = (4,4))
    # cl_img = clahe.apply(simg)

    equ = ocv.equalizeHist(simg)

    # final = 255 - equ 
    # bw = equ > np.mean(equ)

    ret, thresh_bw = ocv.threshold(equ,np.mean(equ),255,ocv.THRESH_BINARY_INV + ocv.THRESH_OTSU)
    #  dilation 
    kernel = np.ones((3,3),np.uint8)
    dil = ocv.dilate(thresh_bw,kernel,iterations=1)
    label_image = label(simg)



    
    # mask = np.sum(cleaned == 255)
    # inf_pix_count = np.sum(thresh_bw == 255)
    # print('infected pixels',inf_pix_count)

    m_img = np.array(cleaned, dtype = float) 
    m_img = (m_img - m_img.min()) / (m_img.max() - m_img.min()) * 255.0  
    m_img = m_img.astype(np.uint8)

    covid_img = m_img - dil
    k = np.ones((3,3),np.uint8)


  


    # white_px = np.sum(m_img == 255 )
    # infected_px = np.sum(dil == 255)

    # print('white mask  pixels',white_px)
    # print('white infected  pixels',infected_px)


    #     # applying countour detection

    # covid = white_px - infected_px
    # print('covid detected',covid )

    # per_covid = (covid / white_px) * 100
    # print('percentage of covid detectin',per_covid)
    # if per_covid <25:
    #   print('NORMAL')
    # else:
    #   print('COVID')



    
        # applying countour detection
    # infected_pixels = number_of_white_pix = np.sum(dil == 255)
    # mask_pixels = number_of_white_pix = np.sum(bin_mask == 255)
    # print('infect pixels', infected_pixels)
    # print('mask pixels', mask_pixels)
    # covid = mask_pixels - infected_pixels
    # print('covid detected',covid )

    # per_covid = (covid/mask_pixels)*100
    # print('percentage of covid detectin',per_covid)
    # if per_covid <25:
    #   print('NORMAL')
    # else:
    #   print('COVID')




    fig, ax = plt.subplots(3, 3, figsize=[12, 12])
    ax[0, 0].set_title("Original")
    ax[0, 0].imshow(img, cmap='gray')
    ax[0, 0].axis('off')
    ax[0, 1].set_title("Threshold")
    ax[0, 1].imshow(binary_image, cmap='gray')
    ax[0, 1].axis('off')
    ax[0, 2].set_title("clear border")
    ax[0, 2].imshow(clear_bw, cmap='gray')
    ax[0, 2].axis('off')
    ax[1, 0].set_title(" after dilation and removing small holes")
    ax[1, 0].imshow(cleaned,cmap='gray')
    ax[1, 0].axis('off')
    ax[1, 1].set_title("mask")
    ax[1, 1].imshow(mask, cmap='gray')

    ax[1, 1].axis('off')
    ax[1, 2].set_title("histogram equalization")
    ax[1, 2].imshow(equ, cmap='gray')


    ax[1, 2].axis('off')
    ax[2, 0].set_title("fill")
    ax[2, 0].imshow(dil, cmap = 'gray')

    ax[2, 0].axis('off')
    ax[2, 1].set_title("infected area")
    ax[2, 1].imshow(covid_img, cmap='gray')

    # ax[2, 1].axis('off')
    # ax[2, 2].set_title("unknown")
    # ax[2, 2].imshow(unknown, cmap='gray')
    

    plt.show()
interact(printer, x=IntSlider(0, 0, len(sample.scan_stack)-1, 1, layout=Layout(width='1500px')))

interactive(children=(IntSlider(value=0, description='x', layout=Layout(width='1500px'), max=587), Output()), …

<function __main__.printer>

In [ ]:
#  libraries for convolutional neural networks 

from keras.layers 